In [28]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [29]:
def time_method(df):
    timestamp=df.iloc[:,0]
    df['month']=timestamp.dt.strftime('%m').astype(np.float64)
    
    df['day']=timestamp.dt.strftime('%d').astype(np.float64)
   
    df['hour']=timestamp.dt.strftime('%H').astype(np.float64)
   
    df['minute']=timestamp.dt.strftime('%M').astype(np.float64)
    
    return df
#only test for continuous data
def miss_forest(x_train,y_train,x_test,dtype):
    if dtype == 'Continious':
        model = RandomForestRegressor()
    elif dtype == 'Categorical':
        model = RandomForestClassifier()
    #gpu can not work for sklearn
    #model.cuda()
    #x_train = x_train.cuda()
    #y_train = y_train.cuda()
    model.fit(x_train,y_train)
    
    y_pred = model.predict(x_test)
    return y_pred


In [33]:

# data preparation for 1.9w data
import random
count = 1
path = 'energydata_complete.csv'
path_miss = 'energydata_complete.csv'
label = 'Appliances'
miss_num = 14

# read data
df = pd.read_csv(path).dropna()
df_miss = pd.read_csv(path_miss)
#print("read from:"+str(len(df_miss)))

#try dropping date
df=df.drop(['date'],axis=1)
df_miss=df_miss.drop(['date'],axis=1)
df_imputed = df_miss.copy()
#print(len(df_imputed))

#select missing columns and set it as nan
header_list = list(df_miss.columns.values)
header_list.remove(label)
predict_cols = random.sample(range(0,len(header_list)),miss_num)

#impute df_miss with df mean
for i in range(len(predict_cols)):
    miss_col = header_list[predict_cols[i]]
    #print('miss feature:'+str(miss_col)+' and imputed by '+ str(np.mean(df[miss_col])))
    df_miss[miss_col].values[:] = np.mean(df[miss_col])

print("data preperation finished")


data preperation finished


In [34]:
"""
path = 'household_power_consumption.txt'
        
df = pd.read_csv(
    path, 
    sep=';', 
    parse_dates={'dt' : ['Date', 'Time']}, 
    infer_datetime_format=True, 
    low_memory=False, 
    na_values=['nan','?'], 
    index_col='dt'
)
#print('Total data size from csv file: '+ str(len(df)))
df_resample = df.resample('30T').mean() # 30 mins resample
print('Data size after resample: '+ str(df_resample.shape[0]))
#print(df_resample.shape[1])
print(df_resample.info())
X = df_resample.dropna().copy() # drop missing data rows, X contains all feature values
y = X['Global_active_power'].shift(-1).ffill()
X = X.drop('Global_active_power', axis=1) #  remove y column
#print(X.info())
print('Data size after dropping missing value rows: '+str(X.shape[0]))
#self.columns = X.columns
"""

"\npath = 'household_power_consumption.txt'\n        \ndf = pd.read_csv(\n    path, \n    sep=';', \n    parse_dates={'dt' : ['Date', 'Time']}, \n    infer_datetime_format=True, \n    low_memory=False, \n    na_values=['nan','?'], \n    index_col='dt'\n)\n#print('Total data size from csv file: '+ str(len(df)))\ndf_resample = df.resample('30T').mean() # 30 mins resample\nprint('Data size after resample: '+ str(df_resample.shape[0]))\n#print(df_resample.shape[1])\nprint(df_resample.info())\nX = df_resample.dropna().copy() # drop missing data rows, X contains all feature values\ny = X['Global_active_power'].shift(-1).ffill()\nX = X.drop('Global_active_power', axis=1) #  remove y column\n#print(X.info())\nprint('Data size after dropping missing value rows: '+str(X.shape[0]))\n#self.columns = X.columns\n"

In [35]:
ite = 0
#while ite < 5:
for j in range(len(predict_cols)):
    miss_col = header_list[predict_cols[j]]
    x_train = df.loc[:,df.columns != miss_col]
    y_train = df[miss_col]
    x_test = df_miss.loc[:,df.columns != miss_col]
    y_test = df_miss[miss_col]

    y_imputed = miss_forest(x_train,y_train,x_test,dtype = 'Continious')
    #df_imputed[miss_col].values[:] = y_imputed
    df_miss[miss_col].values[:] = y_imputed
    loss = abs(y_imputed-y_train)/y_train
    print(f'miss feature: {miss_col} loss: {np.mean(loss):10.8f}')
    
ite += 1
print('iteration: '+str(ite))
#print(df_imputed.head(5))
print(df_miss.head(5))

file_name = 'imputed_'+str(len(predict_cols))+'_'+ path_miss
#df_imputed.to_csv(file_name)
df_miss.to_csv(file_name)
print(file_name+' is saved!')
    

miss feature: Visibility loss: 0.28983874
miss feature: RH_7 loss: 0.04511347
miss feature: lights loss:        inf
miss feature: RH_2 loss: 0.03672654
miss feature: T5 loss: 0.03719749
miss feature: Press_mm_hg loss: 0.00451301
miss feature: rv1 loss: 0.00028382
miss feature: T2 loss: 0.07837135
miss feature: T1 loss: 0.03133340
miss feature: T6 loss:        inf
miss feature: Tdewpoint loss:        inf
miss feature: RH_9 loss: 0.02373763
miss feature: RH_out loss: 0.08254803
miss feature: T7 loss: 0.01237309
iteration: 1
   Appliances  lights         T1       RH_1         T2       RH_2     T3  \
0          60       4  21.361550  47.596667  20.918008  44.453394  19.79   
1          60       4  21.398450  46.693333  20.907900  44.474569  19.79   
2          50       2  21.386017  46.300000  20.915675  44.430625  19.79   
3          50       2  21.382583  46.066667  20.925850  44.416258  19.79   
4          60       4  21.371983  46.333333  20.931000  44.396356  19.79   

        RH_3   